In [ ]:
import pprint as pp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as col
import numpy as np
import json
# import geopandas as gpd
# import geoplot as gpt

from shapely.geometry import Point, Polygon
from dotenv import load_dotenv

import requests
import os
import math
import pandas_gbq

load_dotenv()

# --------- OSM ---------

from OSMPythonTools.api import Api  
from OSMPythonTools.nominatim import Nominatim # Tool to search OSM data by name and address
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass # read-only API that serves up custom selected parts of the OSM map data


In [ ]:

api = Api()
nominatim  = Nominatim()
overpass = Overpass()

#AREA ID
BENIN_ID = 3600192784
BORGOU_ID = 3602803880
NIKKI_COM_ID = 3602859963
NIKKI_ARR_ID = 3611176398

#RELATION ID
BENIN_ID_REL = 192784
BORGOU_ID_REL = 2803880
NIKKI_ID_REL = 2859963

#CONSTANTS
DEG_to_KM2 = 13000
DEG_to_KM = 130

def getAreaInOSM(osm_id, type):
  if type == 'way':
      return int(osm_id) + 2400000000
  if type == 'relation':
      return int(osm_id) + 3600000000
  return ''

def updateAreaIdOfDataframe(df):
  for index, row in df.iterrows():
    if row['area_id'] == '' or int(row['area_id']) == 0:
      df.at[index,'area_id'] = getAreaInOSM(row['osm_id'], row['type'])

def countBuildingsInAreaID(area_id):
  buildings_query = overpassQueryBuilder(area=area_id, elementType=['way'], selector="building" , out='count')
  buildings_res = overpass.query(buildings_query)
  return buildings_res.countWays()

def getBuildingsInVillage(village_df):
  for index, village_row in village_df.iterrows():
    nb_buildings = 0
    density = 0

    # Check if there have already been registered
    if int(village_row['nb_buildings']) < 0:
      nb_buildings = countBuildingsInAreaID(village_row['area_id'])

      # Update the density 
      if nb_buildings > 0:
        density = nb_buildings / (float(str(village_row['area']).replace(',','.')) * 1000000.0)

      df.loc[index, 'nb_buildings'] = nb_buildings
      df.loc[index, 'build_dens'] = density
      df.loc[index, 'pop_est'] = nb_buildings * 5

# ___DISTRICTS___ 

def saveDistricts(district_df):
  destination_table = 'osm_data.districts'
  district_df = district_df.drop(columns=['polygon'],axis=1)
  pandas_gbq.to_gbq(district_df, project_id='sidhouses', destination_table=destination_table , if_exists='replace')

def loadDistricts():
  df = pandas_gbq.read_gbq('SELECT * FROM sidhouses.osm_data.districts')
  create_polygons(df)
  return df

# ___VILLAGES___ 

def saveVillages(village_df):
    destination_table = 'osm_data.villages'
    village_df = village_df.drop(columns=['polygon','point'],axis=1)
    pandas_gbq.to_gbq(village_df, project_id='sidhouses', destination_table=destination_table , if_exists='replace')

def loadVillages():
    df = pandas_gbq.read_gbq('SELECT * FROM sidhouses.osm_data.villages')
    create_polygons(df)
    create_points(df)
    return df

# ___BUILDINGS___ 

def saveBuildings(buildings_df):
    destination_table = 'osm_data.buildings'
    buildings_df = buildings_df.drop(columns=['polygon','point'],axis=1)
    pandas_gbq.to_gbq(buildings_df, project_id='sidhouses', destination_table=destination_table , if_exists='replace')

def loadBuildings():
    df = pandas_gbq.read_gbq('SELECT * FROM sidhouses.osm_data.buildings')
    create_polygons(df)
    create_points(df)
    return df


# --------- AUX ---------

def create_polygons(df):
  df['polygon'] = ""
  for index, row in df.iterrows():
    if (row['boundary_lon'] != '' and  row['boundary_lat'] != ''):
      lon = json.loads(row['boundary_lon'])
      lat = json.loads(row['boundary_lat'])
      df.at[index,'polygon'] = Polygon(zip(lon,lat))

def create_points(df):
  df['point'] = ""
  for index, row in df.iterrows():
    if (row['lat'] != '' and  row['lon'] != ''):
      p = Point(float(row['lon']) , float(row['lat']))
      df.at[index,'point'] = p 

def isIdalready(df, _id):
  for index, row in df.iterrows():
    if int(row['osm_id']) == int(_id):
      return True
  return False


def getDistrictOfVillage(district_df, village_df):
  # Loop throught all villages
  for i, village_row in village_df.iterrows():
    # Check if there is already a district
    if village_row['district'] == '':

      # Loop throught all districts to see if the village is inside the polygon
      for j, district_row in district_df.iterrows():
        point = village_row['point']
        isin = district_row['polygon'].contains(point)
        if isin == True :
          village_df.at[i,'district'] = district_row['name']
          break


In [ ]:
### **Districts of Nikki** (County)

In [ ]:
# Plot the districts 
district_df = loadDistricts();

district_geodf = gpd.GeoDataFrame(district_df, geometry="polygon");
district_geodf.rename(columns={"polygon": "geometry"}, inplace=True);
fig, ax = plt.subplots(1, 1,figsize=(15,15))
district_geodf.plot(column="name",  ax=ax, legend=False, cmap='Blues', figsize=(15,15));

In [ ]:
### **Villages in Nikki** (County)

In [ ]:
village_query = overpassQueryBuilder(area=NIKKI_COM_ID, elementType='way', selector="'place'~'village|locality|town|city|hamlet'", out="body geom")
residential_query = overpassQueryBuilder(area=NIKKI_COM_ID, elementType='way', selector="'landuse'~'residential'", out="body geom")

village_res = overpass.query(village_query)
residential_res = overpass.query(residential_query)

# Transform response into json 
village_json = village_res.toJSON()
residential_res = residential_res.toJSON()

village_headers = ['osm_id','area_id','type','place','district','name','link','lat','lon','perim','area','boundary_lat','boundary_lon','polygon','point','nb_buildings','building_density','population_est_building']
new_village_data = []

# Loop throught the elements in the response
for ele in village_json['elements']:
    geo_df = pd.DataFrame.from_records(ele['geometry'])
    poly = Polygon(zip(geo_df['lon'], geo_df['lat']))
    point = Point(poly.centroid.x,poly.centroid.y)
    boudary_lat =  str(geo_df['lat'].tolist())
    boudary_lon =  str(geo_df['lon'].tolist())
    ele_row = [ 
               ele.get('id',0),                    # osm_id
               0,                                  # area_id
               ele.get('type',''),                 # type
               ele['tags'].get('place',''),        # place
               '',                                 # district
               ele['tags'].get('name',''),         # name
               'https://www.openstreetmap.org/edit?' + str(ele.get('type','')) + '=' + str(ele.get('id','')),  # link
               str(poly.centroid.y),               # lat
               str(poly.centroid.x),               # lon
               str(poly.length * DEG_to_KM),       # perim
               str(poly.area * DEG_to_KM2),        # area
               boudary_lat,                        # boudary_lat
               boudary_lon,                        # boudary_lon
               poly,                               # polygon
               point,                              # point
               -1,                                 # nb_buildings
               0.0,                                # building_density
               0.0,                                # population_est_building
               ]
   
    new_village_data.append(ele_row)

# Loop throught the elements in the response
for ele in residential_res['elements']:
    geo_df = pd.DataFrame.from_records(ele['geometry'])
    poly = Polygon(zip(geo_df['lon'], geo_df['lat']))
    point = Point(poly.centroid.x,poly.centroid.y)
    boudary_lat =  str(geo_df['lat'].tolist())
    boudary_lon =  str(geo_df['lon'].tolist())
    ele_row = [ 
               ele.get('id',0),                    # osm_id
               0,                                  # area_id
               ele.get('type',''),                 # type
               ele['tags'].get('landuse',''),      # landuse
               '',                                 # district
               ele['tags'].get('name',''),         # name
               'https://www.openstreetmap.org/edit?' + str(ele.get('type','')) + '=' + str(ele.get('id','')),  # link
               str(poly.centroid.y),               # lat
               str(poly.centroid.x),               # lon
               str(poly.length * DEG_to_KM),       # perim
               str(poly.area * DEG_to_KM2),        # area
               boudary_lat,                        # boudary_lat
               boudary_lon,                        # boudary_lon
               poly,                               # polygon
               point,                              # point
               -1,                                 # nb_buildings
               0.0,                                # building_density
               0.0,                                # population_est_building
               ]
   
    new_village_data.append(ele_row)

# Create Dataframe from data
new_village_df = pd.DataFrame(new_village_data,columns=village_headers)
print(new_village_df.shape)

In [ ]:
# Load villages
village_df = loadVillages()
village_df.shape

In [ ]:
# Update missing values in dataframe from spreadsheet
for index, new_row in new_village_df.iterrows():
  if not isIdalready(village_df,new_row['osm_id']) :
    village_df = village_df.append(new_row)

# Update missing areas id
updateAreaIdOfDataframe(village_df)

# Update missing buildings numbers
# getBuildingsInVillage(village_df)

# Update county origin
getDistrictOfVillage(district_df, village_df)

In [ ]:
### **Buildings of Nikki** (County)

In [ ]:
villages_selected_id = [
    377129182, # MONNON
    737083662, # SANSI GANDÓ
    738644153, # BESEN GOUROU
    738657269, # BARKEDJE BARIKEDJE
    745997907, # BOUDAL
]

village_df.loc[village_df['osm_id'] == 377129182]

In [ ]:
def get_buildings_in_village(village_row_df):

    building_query = overpassQueryBuilder(area=village_row_df['area_id'].values[0], elementType='way', selector="building", out="geom")

    building_res = overpass.query(building_query)

    # Transform response into json 
    building_json = building_res.toJSON()
    new_building_data = []

    # Loop throught the elements in the response
    for ele in building_json['elements']:
        geo_df = pd.DataFrame.from_records(ele['geometry'])
        poly = Polygon(zip(geo_df['lon'], geo_df['lat']))
        point = Point(poly.centroid.x,poly.centroid.y)
        boudary_lat =  str(geo_df['lat'].tolist())
        boudary_lon =  str(geo_df['lon'].tolist())
        ele_row = [ 
                   ele.get('id',0),                    # osm_id
                   ele.get('type',''),                 # type
                   village_row_df['district'].values[0],         # district
                   village_row_df['name'].values[0],             # village
                   str(poly.centroid.y),               # lat
                   str(poly.centroid.x),               # lon
                   str(poly.length * DEG_to_KM),       # perim
                   str(poly.area * DEG_to_KM2),        # area
                   boudary_lat,                        # boudary_lat
                   boudary_lon,                        # boudary_lon
                   poly,                               # polygon
                   point,                              # point
                   ]
    
        new_building_data.append(ele_row)
    return new_building_data

building_headers = ['osm_id','type','district','village','lat', 'lon','perim','area','boundary_lat','boundary_lon','polygon', 'point']
new_buildings_data = []
for vill_id in villages_selected_id:
    row = village_df.loc[village_df['osm_id'] == vill_id ]
    buildings = get_buildings_in_village(row)
    new_buildings_data.extend(buildings)

# Create Dataframe from data
new_building_df = pd.DataFrame(new_buildings_data,   columns=building_headers)
new_building_df.shape

In [ ]:
# Load buildings
building_df = loadBuildings()
building_df.shape

In [ ]:
# Update missing values in dataframe from spreadsheet
for index, new_row in new_building_df.iterrows():
  if not isIdalready(building_df,new_row['osm_id']) :
    building_df = building_df.append(new_row)

building_df.shape

In [ ]:
# Save buildings
saveBuildings(building_df)